In [1]:
%env CUDA_VISIBLE_DEVICES= 3,5,7


env: CUDA_VISIBLE_DEVICES=3,5,7


In [2]:
import os
os.environ['HF_HOME'] = '/home/sofia/cache_custom'

## IndicTrans2

In [19]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransToolkit import IndicProcessor
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm import tqdm
from torch.nn.functional import softmax


BATCH_SIZE = 16 # edited from 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None
print(DEVICE)

cuda


In [4]:
import transformers
print(transformers.__version__)

4.44.2


In [5]:
import importlib
import possible_indic_relations as poss_indic_rel
# Reload the module to reflect changes
importlib.reload(poss_indic_rel)

pir= poss_indic_rel.possible_relations
pir

ambiguos_words = list(pir.keys())

In [6]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model

In [7]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir,  quantization)

ip_en_ind = IndicProcessor(inference=True)

In [8]:


def resolve_logits_for_best_beam(outputs, num_beams):
    """ Resolve the logits from the best beam, using model output from a generate call.
        For a shape [tokens?, batch_size*num_beams, vocab], returns [tokens?, batch_size, vocab]

        Assumes num_return_sequences=1."""

    # print("length of output beam", len(outputs.beam_indices))
    # print("shape of beam_indices", outputs.beam_indices.shape)
    # print("shape of logits", (outputs.logits[0].shape))
    # print("length of logits", len(outputs.logits))
    # print("length od outputs", len(outputs))
    best_logits  = []
    beam_indices = [ outputs.beam_indices[:,i].tolist() for i in range(len(outputs.logits)-1) ]
    # print("length of beam_indices", len(beam_indices))

    for beam_index, logits in zip(beam_indices, outputs.logits):
        beam_index = [ idx if idx != -1 else ((num_beams*(i+1))-1) for i, idx in enumerate(beam_index) ]
        best_logits.append(logits[beam_index,:])

    return best_logits




def get_logits_for_span(logits, translations, tokenizer, search_spans):
    """ Given search spans, returns the logits before the span was generated.

    Args:
        logits (tuple[Tensor]): Tuple of tensors, of shape [tokens?, batch_size, vocab]
        sequences (tuple[list[int]]): Tokenized output sequences.
        tokenizer (PreTrainedTokenizerBase): Tokenizer for the model.
        search_spans (list[str]): batch_size spans to search for. Must be present in the generated sequences.

    Returns:
        Tensor: Tensor of shape [batch_size, vocab] indicating the logits before the span for each batch element.
    """
    if isinstance(search_spans, str):
        search_spans = [ search_spans ] * len(translations)

    # with tokenizer.as_target_tokenizer():   
    #     detok_outputs = tokenizer.batch_decode(translations, skip_special_tokens=True)

    # positions = [ output.index(span) for output, span in zip(translations, search_spans) ]
    logit_pos = [  ]

    for seq,  span,  in zip(translations,  search_spans): 
        subtokens = []
        encodes=[]
        with tokenizer.as_target_tokenizer():
            subtokens = tokenizer(span, add_special_tokens=False, ).input_ids
            encodes = tokenizer.encode(span, add_special_tokens=False, return_tensors="pt")
            print("encodes", encodes)
        # subtokens = tokenizer(span, add_special_tokens=False).input_ids
        encodes = tokenizer.encode(span, add_special_tokens=False, return_tensors="pt")
        print("encode::::", encodes)
        print("subtokens", subtokens)
        print("for span:", span, "subtokens", subtokens)
        print("seq", seq)
        idx = 0
        while idx < len(seq):
            if all(seq[idx+i] == tok for i, tok in enumerate(subtokens)): 
                print( "found", idx)
                break
            idx += 1
        logit_pos.append(idx-1)
    
    print("logit_pos", logit_pos)
    print("Dimensions of logits", logits[0].shape, len(logits))

    selected_logits = []
    # Iterate over each batch and corresponding token position
    for batch, token in enumerate(logit_pos):
        # print("batch, token", batch, token)
        # print("len(logits)", len(logits))
        # print("Shape of logits", logits[0].shape)
        # # print("logits token", logits[token])
        # print("logits 1st item 1st row",logits[0][0])
        # Extract logits for the specific token position in the current batch
        current_logit = logits[token][batch, :]
        # current_logit = logits[token][0]
        
        # Append the selected logit to the list
        selected_logits.append(current_logit)

    # Stack the list of selected logits into a tensor
    selected_logits = torch.stack(selected_logits)

    return selected_logits
        
    # return selected_logits
    # return torch.stack([ logits[token][batch,:] for batch, token in enumerate(logit_pos) ])


In [9]:
def get_search_spans(inp_sents, tgt_lang, translations):
    search_spans= []
    for idx, inp in enumerate(inp_sents):
        # check which word from ambiguos_Wwords in present in the input sentence
        for word in ambiguos_words:
            if word in inp:
                curr_amb_word= word

        # get the possible relations for the current ambiguous word
        possible_relations= pir[curr_amb_word][tgt_lang].keys()
        # print("Possible relations for the word", curr_amb_word, "in", tgt_lang, "are", possible_relations)

        # find the word in the translation from the possible relations. if not found print the translation
        for rel in possible_relations:
            if rel in translations[idx]:
                # print("Relation found in the translation for the word", curr_amb_word, "in", tgt_lang, "in the sentence", translations[idx], "at ", translations[idx].index(rel), "\nso sentece is", translations[idx][translations[idx].index(rel):])
                search_spans.append(rel)
                break
        else:
            print("No relation found in the translation for the word", curr_amb_word, "in", tgt_lang, "in the sentence", translations[idx])
            search_spans.append(translations[idx][0])
    print("Search spans are", search_spans)
    return search_spans
       



In [10]:



def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    start_logits = []
    for i in tqdm(range(0, len(input_sentences), BATCH_SIZE)):
        batch = input_sentences[i : i + BATCH_SIZE]
        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)
        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)
        with torch.no_grad():
            # generated_tokens = model.generate(
            outputs = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1, # TODO temp
                output_scores=True,
                output_logits=True,
                return_dict_in_generate=True,

            )
            print("Length of outputs.logits actual", len(outputs.logits))
            print("Shape of outputs.logits actual", outputs.logits[0].shape)

            print("Length of outputs.beam_indices actual", len(outputs.beam_indices))
            print("Shape of outputs.beam_indices actual", outputs.beam_indices.shape)
            
            outputs.beam_indices = outputs.beam_indices.cpu()
            outputs.logits = tuple(logits.cpu() for logits in outputs.logits)               
        # Decode the generated tokens into text
        generated_tokens = outputs.sequences
        # print("len generated_tokens: ", (generated_tokens[0]).shape)
        print("1st generated token: ", generated_tokens[0])
        vector = generated_tokens.detach().cpu().tolist()
        # print("length of outputs vectors: ", len(vector), len(vector[0]))
        # print("vector of generated_tokens: ", vector)
        print("1st vector: ", vector[0])



        with tokenizer.as_target_tokenizer():
            decoded_op = tokenizer.batch_decode(
                vector,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        print("1st decoded_op: ", decoded_op[0])
        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(decoded_op, lang=tgt_lang)

        print("translations: ", translations)
        

        search_spans = get_search_spans(batch,  tgt_lang, translations)
        print("length search_spans: ", len(search_spans))
        best_logits = resolve_logits_for_best_beam(outputs, num_beams=5)
        # print("length best_logits: ",len(best_logits))
        print("length of best_logits: ", len(best_logits), (best_logits[0]).shape)

        start_logits += get_logits_for_span(best_logits, outputs.sequences, tokenizer, search_spans)
        # start_logits += get_logits_for_span(best_logits, translations, tokenizer, search_spans)

        del inputs
        torch.cuda.empty_cache()

    return translations, start_logits

In [9]:
lang_script_list = [
                           'ory_Orya',
                     'pan_Guru', 'ben_Beng', 
                       'mal_Mlym',
                           'mar_Deva', 
                           'tam_Taml', 'guj_Gujr', 
                           'tel_Telu', 'hin_Deva', 
                           'kan_Knda', 
                           ]

In [12]:
test_csv_folder = 'custom_test_csv'

In [13]:
lang_code_map = {
    'eng_Latn': 'en',
    'hin_Deva': 'hi',
    'guj_Gujr': 'gu',
    'kan_Knda': 'kn',
    'mal_Mlym': 'ml',
    'mar_Deva': 'mr',
    'tam_Taml': 'ta',
    'tel_Telu': 'te',
    'pan_Guru': 'pa',
    'ben_Beng': 'bn',
    'ory_Orya': 'or'
}


In [14]:
sents = []
with open('test_sentences_eng.txt', 'r') as f:
    sents = f.readlines()
sents = [sent.strip() for sent in sents if sent.find('child')==-1]
print(len(sents))

1809


In [15]:
sents = sents[:10]
sents

['My grandmother is a Brahmin.',
 'My grandfather is a Brahmin.',
 'My uncle is a Brahmin.',
 'My aunt is a Brahmin.',
 'My brother-in-law is a Brahmin.',
 'My sister-in-law is a Brahmin.',
 'My cousin is a Brahmin.',
 'My nephew is a Brahmin.',
 'My niece is a Brahmin.',
 'My grandmother is a Kshatriya.']

In [10]:
# print("PIR ambiguos words:", ambiguos_words)
words_ids={}
for lang in lang_script_list:
    words_ids[lang]={}
    # print("Translating to", lang, words_ids)
    for word in ambiguos_words:
        # pir[word][lang].keys will be the key of words_ip map and value will be -1
        for key in pir[word][lang].keys():
            words_ids[lang][key]=-1

        

print("Words_ids:", words_ids)

       

Words_ids: {'ory_Orya': {'ଜେଜେମା': -1, 'ଆଈ': -1, 'ଜେଜେବାପା': -1, 'ଅଜା': -1, 'ବଡ଼ବାପା': -1, 'ଦାଦା': -1, 'ମାମୁଁ': -1, 'ପିଉସା': -1, 'ମଉସା': -1, 'ପିଉସୀ': -1, 'ମାଉସୀ': -1, 'ମାଇଁ': -1, 'ବଡ଼ମାଆ': -1, 'ଖୁଡ଼ି': -1, 'ବଡ଼ ଶଳା': -1, 'ଶଳା': -1, 'ଭିଣେଇ': -1, 'ଭିଣୋଇ': -1, 'ଦେଢ଼ଶୁର': -1, 'ଦିଅର': -1, 'ବଡ଼ ନଣନ୍ଦ': -1, 'ନଣନ୍ଦ': -1, 'ଭାଉଜ': -1, 'ଭାଇବୋହୁ': -1, 'ଦେଢ଼ଶାସୁ': -1, 'ଶାଳୀ': -1, 'ଭାଇ': -1, 'ଦିଦି': -1, 'ପୁଅ': -1, 'ଝିଅ': -1, 'ପିଲା': -1, 'ପୁତୁରା': -1, 'ଭଣଜା': -1, 'ଝିଆରୀ': -1, 'ଭାଣିଜୀ': -1}, 'pan_Guru': {'ਦਾਦੀ': -1, 'ਨਾਨੀ': -1, 'ਦਾਦਾ': -1, 'ਨਾਨਾ': -1, 'ਤਾਇਆ': -1, 'ਚਾਚਾ': -1, 'ਮਾਮਾ': -1, 'ਫੁੱਫੜ': -1, 'ਮਾਸੜ': -1, 'ਭੂਆ': -1, 'ਮਾਸੀ': -1, 'ਮਾਮੀ': -1, 'ਤਾਈ': -1, 'ਸਾਲਾ': -1, 'ਜੀਜਾ': -1, 'ਜੇਠ': -1, 'ਦੇਵਰ': -1, 'ਸਾਂਡੂ': -1, 'ਨਣਦ': -1, 'ਭਾਬੀ': -1, 'ਸਾਲੀ': -1, 'ਜੇਠਾਣੀ': -1, 'ਭਾਈ': -1, 'ਭੈਣ': -1, 'ਮੁੰਡਾ': -1, 'ਕੁੜੀ': -1, 'ਭਤੀਜਾ': -1, 'ਭਾਂਜਾ': -1, 'ਭਤੀਜੀ': -1, 'ਭਾਂਜੀ': -1}, 'ben_Beng': {'ঠাকুরমা': -1, 'দিদি মা': -1, 'ঠাকুরদা': -1, 'দাদু': -1, 'জেঠা মশাই': -1, 'কাকু': -1, 'মামা': -1, 'পিশে মশাই': -1, 'মেশো মশাই': -

In [29]:
# word_trl=[]
span_encodings = {}
for lang in lang_script_list:
#   if lang == 'ory_Orya':
    span_encodings[lang] = {}
    inputs = ambiguos_words
    for i in tqdm(range(0, len(inputs), BATCH_SIZE)):
        batch = inputs[i : i + BATCH_SIZE]
        print("Batch:", batch)  

        # batch = ip_en_ind.preprocess_batch(words_ids[lang].keys().tolist(), src_lang=lang, tgt_lang=lang)
        batch = ip_en_ind.preprocess_batch(batch, src_lang='eng_Latn', tgt_lang=lang)
        print("Batch:", batch)
        # # Tokenize the batch and generate input encodings
        inputs = en_indic_tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        with torch.no_grad():
            # generated_tokens = model.generate(
            outputs = en_indic_model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1, # TODO temp
                output_scores=True,
                output_logits=True,
                return_dict_in_generate=True,

            )
            # print("Length of outputs.logits actual", len(outputs.logits))
            # print("Shape of outputs.logits actual", outputs.logits[0].shape)

            # print("Length of outputs.beam_indices actual", len(outputs.beam_indices))
            # print("Shape of outputs.beam_indices actual", outputs.beam_indices.shape)
            
            outputs.beam_indices = outputs.beam_indices.cpu()
            outputs.logits = tuple(logits.cpu() for logits in outputs.logits)               
        # Decode the generated tokens into text
        generated_tokens = outputs.sequences
        # print("len generated_tokens: ", (generated_tokens[0]).shape)
        print("1st generated token: ", generated_tokens[0])
        vector = generated_tokens.detach().cpu().tolist()
        # print("length of outputs vectors: ", len(vector), len(vector[0]))
        # print("vector of generated_tokens: ", vector)
        print("1st vector: ", vector[0])
        


        with en_indic_tokenizer.as_target_tokenizer():
            decoded_op = en_indic_tokenizer.batch_decode(
                vector,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        print("1st decoded_op: ", decoded_op[0])
        # Postprocess the translations, including entity replacement
        word_trl = ip_en_ind.postprocess_batch(decoded_op, lang=lang)

        print("translations: ", word_trl)
        for word in word_trl:
            word_index = word_trl.index(word)
            if word_index < len(vector):
                span_encodings[lang][word] = vector[word_index]
            else:
                print(f"Index {word_index} out of range for vector of length {len(vector)}")


span_encodings



  0%|          | 0/1 [00:00<?, ?it/s]

Batch: ['grandmother', 'grandfather', 'uncle', 'aunt', 'brother-in-law', 'sister-in-law', 'cousin', 'child', 'nephew', 'niece']
Batch: ['eng_Latn ory_Orya grandmother', 'eng_Latn ory_Orya grandfather', 'eng_Latn ory_Orya uncle', 'eng_Latn ory_Orya aunt', 'eng_Latn ory_Orya brother-in-law', 'eng_Latn ory_Orya sister-in-law', 'eng_Latn ory_Orya cousin', 'eng_Latn ory_Orya child', 'eng_Latn ory_Orya nephew', 'eng_Latn ory_Orya niece']


100%|██████████| 1/1 [00:00<00:00,  2.46it/s]


1st generated token:  tensor([    2, 41445,   241,    30,     2,     1,     1,     1,     1],
       device='cuda:0')
1st vector:  [2, 41445, 241, 30, 2, 1, 1, 1, 1]
1st decoded_op:  जेजेमा'
translations:  ["ଜେଜେମା'", 'ଜେଜେବାପା ', 'ମାମୁଁ। ', 'ମାଉସୀ। ', 'ଶ୍ୱଶୁର-ଶ୍ୱଶୁର ', 'ଶ୍ୱଶୁର। ', 'ସମ୍ପର୍କୀଯ଼ ଭାଇ। ', 'ଶିଶୁ ', 'ପୁତୁରା ', 'ଭାଣିଜୀ ']


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Batch: ['grandmother', 'grandfather', 'uncle', 'aunt', 'brother-in-law', 'sister-in-law', 'cousin', 'child', 'nephew', 'niece']
Batch: ['eng_Latn kan_Knda grandmother', 'eng_Latn kan_Knda grandfather', 'eng_Latn kan_Knda uncle', 'eng_Latn kan_Knda aunt', 'eng_Latn kan_Knda brother-in-law', 'eng_Latn kan_Knda sister-in-law', 'eng_Latn kan_Knda cousin', 'eng_Latn kan_Knda child', 'eng_Latn kan_Knda nephew', 'eng_Latn kan_Knda niece']


100%|██████████| 1/1 [00:00<00:00,  3.46it/s]

1st generated token:  tensor([    2,  4565, 35330,     2,     1,     1], device='cuda:0')
1st vector:  [2, 4565, 35330, 2, 1, 1]
1st decoded_op:  अज्जि 
translations:  ['ಅಜ್ಜಿ ', 'ಅಜ್ಜ ', 'ಚಿಕ್ಕಪ್ಪ ', 'ಚಿಕ್ಕಮ್ಮ. ', 'ಸೋದರ ಸಂಬಂಧಿ ', 'ಅತ್ತಿಗೆ ', 'ಸೋದರಸಂಬಂಧಿ ', 'ಮಗು. ', 'ಸೋದರಳಿಯ ', 'ಸೋದರ ಸೊಸೆ ']


{'ory_Orya': {"ଜେଜେମା'": [2, 41445, 241, 30, 2, 1, 1, 1, 1],
  'ଜେଜେବାପା ': [2, 41445, 1007, 1714, 2, 1, 1, 1, 1],
  'ମାମୁଁ। ': [2, 9971, 19212, 6, 2, 1, 1, 1, 1],
  'ମାଉସୀ। ': [2, 30261, 694, 6, 2, 1, 1, 1, 1],
  'ଶ୍ୱଶୁର-ଶ୍ୱଶୁର ': [2, 21405, 699, 22252, 13, 21405, 699, 22252, 2],
  'ଶ୍ୱଶୁର। ': [2, 21405, 699, 22252, 6, 2, 1, 1, 1],
  'ସମ୍ପର୍କୀଯ଼ ଭାଇ। ': [2, 60824, 3991, 6, 2, 1, 1, 1, 1],
  'ଶିଶୁ ': [2, 3442, 2, 1, 1, 1, 1, 1, 1],
  'ପୁତୁରା ': [2, 4300, 5686, 2, 1, 1, 1, 1, 1],
  'ଭାଣିଜୀ ': [2, 980, 9742, 795, 2, 1, 1, 1, 1]},
 'pan_Guru': {'ਦਾਦੀ ਮਾਂ ': [2, 29498, 640, 2, 1, 1, 1],
  'ਦਾਦਾ ਜੀ ': [2, 15588, 613, 2, 1, 1, 1],
  'ਚਾਚਾ ': [2, 34059, 2, 1, 1, 1, 1],
  'ਮਾਸੀ ਜੀ। ': [2, 65770, 613, 6, 2, 1, 1],
  'ਭਰਾ-ਸੱਸ ': [2, 8327, 13, 116, 19, 115, 2],
  'ਭਰਜਾਈ ': [2, 1144, 40842, 2, 1, 1, 1],
  'ਚਚੇਰਾ ਭਰਾ ': [2, 49615, 2656, 8327, 2, 1, 1],
  'ਬੱਚਾ ': [2, 336, 19, 317, 2, 1, 1],
  'ਭਤੀਜੇ ': [2, 39136, 1386, 2, 1, 1, 1],
  'ਭਤੀਜੀ ': [2, 39136, 795, 2, 1, 1, 1]},
 'ben_Beng': {'দিদিমা ': 

In [23]:
word_transl=[]
words_ids={}

for lang in lang_script_list:
  if lang == 'ory_Orya':
    words_ids[lang]={}
    inputs=[list(pir[word][lang].keys()) for word in ambiguos_words]
    inputs = [item for sublist in inputs for item in sublist]
    print("Inputs:", inputs)
    for i in tqdm(range(0, len(inputs), BATCH_SIZE)):
        batch = inputs[i : i + BATCH_SIZE]
        print("Batch:", batch)  

        # batch = ip_en_ind.preprocess_batch(words_ids[lang].keys().tolist(), src_lang=lang, tgt_lang=lang)
        batch = ip_en_ind.preprocess_batch(batch, src_lang=lang, tgt_lang=lang)
        print("Batch:", batch)
        # # Tokenize the batch and generate input encodings
        # inputs = en_indic_tokenizer(
        #     batch,
        #     truncation=True,
        #     padding="longest",
        #     return_tensors="pt",
        #     return_attention_mask=True,
        # ).to(DEVICE)
        
        # print("Inputs:", inputs)
        # # encodes = en_indic_tokenizer.encode(inputs.input_ids, add_special_tokens=False, return_tensors="pt")
        # # print("encode::::", encodes)
        
        # encodes = en_indic_tokenizer.encode(batch, add_special_tokens=False, return_tensors="pt")
        # print("encode::::", encodes)
        

        print("betch encode plus:", en_indic_tokenizer.batch_encode_plus(batch, return_tensors="pt", padding=True, truncation=True))
        print("encode:", en_indic_tokenizer.encode(batch, add_special_tokens=False, return_tensors="pt"))
        print("encode batch:", en_indic_tokenizer.encode_batch(batch))
        

       

Inputs: ['ଜେଜେମା', 'ଆଈ', 'ଜେଜେବାପା', 'ଅଜା', 'ବଡ଼ବାପା', 'ଦାଦା', 'ମାମୁଁ', 'ପିଉସା', 'ମଉସା', 'ପିଉସୀ', 'ମାଉସୀ', 'ମାଇଁ', 'ବଡ଼ମାଆ', 'ଖୁଡ଼ି', 'ବଡ଼ ଶଳା', 'ଶଳା', 'ଭିଣେଇ', 'ଭିଣୋଇ', 'ଦେଢ଼ଶୁର', 'ଦିଅର', 'ବଡ଼ ନଣନ୍ଦ', 'ନଣନ୍ଦ', 'ଭାଉଜ', 'ଭାଇବୋହୁ', 'ଦେଢ଼ଶାସୁ', 'ଶାଳୀ', 'ଭାଇ', 'ଦିଦି', 'ପୁଅ', 'ଝିଅ', 'ପିଲା', 'ପୁତୁରା', 'ଭଣଜା', 'ଝିଆରୀ', 'ଭାଣିଜୀ']


  0%|          | 0/3 [00:00<?, ?it/s]

Batch: ['ଜେଜେମା', 'ଆଈ', 'ଜେଜେବାପା', 'ଅଜା', 'ବଡ଼ବାପା', 'ଦାଦା', 'ମାମୁଁ', 'ପିଉସା', 'ମଉସା', 'ପିଉସୀ', 'ମାଉସୀ', 'ମାଇଁ', 'ବଡ଼ମାଆ', 'ଖୁଡ଼ି', 'ବଡ଼ ଶଳା', 'ଶଳା']
Batch: ['ory_Orya ory_Orya जेजेमा', 'ory_Orya ory_Orya आई', 'ory_Orya ory_Orya जेजेबापा', 'ory_Orya ory_Orya अजा', 'ory_Orya ory_Orya बड़बापा', 'ory_Orya ory_Orya दादा', 'ory_Orya ory_Orya मामुँ', 'ory_Orya ory_Orya पिउसा', 'ory_Orya ory_Orya मउसा', 'ory_Orya ory_Orya पिउसी', 'ory_Orya ory_Orya माउसी', 'ory_Orya ory_Orya माइँ', 'ory_Orya ory_Orya बड़माआ', 'ory_Orya ory_Orya खुड़ि', 'ory_Orya ory_Orya बड़ शळा', 'ory_Orya ory_Orya शळा']
betch encode plus: {'input_ids': tensor([[    1,     1,    69,    69,   277, 21061, 11027, 21061, 11027, 13622,
          6764,     2],
        [    1,     1,     1,     1,     1,     1,    69,    69,   277, 20157,
         27251,     2],
        [   69,    69,   277, 21061, 11027, 21061, 11027, 21375,  6764, 16446,
          6764,     2],
        [    1,     1,     1,     1,     1,    69,    69,   277, 232

AttributeError: 'IndicTransTokenizer' object has no attribute 'encode_batch'

In [ ]:
src_lang = "eng_Latn"

for lang in lang_script_list:
    tgt_lang = lang
    print(lang)
    translations, logits = batch_translate(sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip_en_ind)


    # # save hindi translations to a file test_translations_hin.txt
    # with open('test_translations/indic_trans2/test_transl_it2_'+lang+'.txt', 'w') as f:
    #     for sent in translations:
    #         f.write(sent + '\n')

    # save the logits to a file 
    with open('test_trasnlations/indic_trans2/test_transl_it2_'+lang+'_logits.txt', 'w') as f:
        for logit in logits:
            f.write(str(logit) + '\n')

ory_Orya


  0%|          | 0/1 [00:00<?, ?it/s]/home/sofia/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00,  2.08it/s]

Length of outputs.logits actual 10
Shape of outputs.logits actual torch.Size([50, 122672])
Length of outputs.beam_indices actual 10
Shape of outputs.beam_indices actual torch.Size([10, 9])
1st generated token:  tensor([    2,   644, 41445,   241,  4725, 28125,     6,     2,     1],
       device='cuda:0')
1st vector:  [2, 644, 41445, 241, 4725, 28125, 6, 2, 1]
1st decoded_op:  मो जेजेमा जणे ब्राह्मण । 
translations:  ['ମୋ ଜେଜେମା ଜଣେ ବ୍ରାହ୍ମଣ। ', 'ମୋ ଜେଜେବାପା ଜଣେ ବ୍ରାହ୍ମଣ। ', 'ମୋ ମାମୁଁ ଜଣେ ବ୍ରାହ୍ମଣ। ', 'ମୋ ମାଉସୀ ଜଣେ ବ୍ରାହ୍ମଣ। ', 'ମୋ ଭିଣୋଇ ଜଣେ ବ୍ରାହ୍ମଣ। ', 'ମୋ ଭାଉଜ ଜଣେ ବ୍ରାହ୍ମଣ। ', 'ମୋର ସମ୍ପର୍କୀଯ଼ ଭାଇ ଜଣେ ବ୍ରାହ୍ମଣ। ', 'ମୋ ପୁତୁରା ଜଣେ ବ୍ରାହ୍ମଣ। ', 'ମୋର ଭାଣିଜୀ ଜଣେ ବ୍ରାହ୍ମଣ। ', 'ମୋ ଜେଜେମା ଜଣେ କ୍ଷତ୍ରିଯ଼। ']
Search spans are ['ଜେଜେମା', 'ଜେଜେବାପା', 'ମାମୁଁ', 'ମାଉସୀ', 'ଭିଣୋଇ', 'ଭାଉଜ', 'ଭାଇ', 'ପୁତୁରା', 'ଭାଣିଜୀ', 'ଜେଜେମା']
length search_spans:  10
length of best_logits:  9 torch.Size([10, 122672])
encodes tensor([[277,   3]])
encode:::: tensor([[277,   3]])
subtokens [590, 103907, 101898, 103907, 1

FileNotFoundError: [Errno 2] No such file or directory: 'test_trasnlations/indic_trans2/test_transl_it2_ory_Orya_logits.txt'

In [ ]:
# # import the test_sentences_eng.txt file data as sents

# # sents = ['The river is blue.', 'My uncle wearing blue coat.', 'My maternal aunt went to the river bank.', "The doctor went to bank for money."]
# sents=['My maternal aunt went to the river bank.', 'My maternal aunt loves me.', 
#        'My maternal aunt loves her grandchildren.', "My maternal aunt loves her children.", 
#        "My maternal aunt loves her family.",
#        ]   
# src_lang = "eng_Latn"

# tgt_lang = 'hin_Deva'
# # print(lang)
# translations, logits = batch_translate(sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip_en_ind)


In [ ]:
print(translations)

In [ ]:
print(logits)

In [ ]:
to break all run

In [ ]:
# for each logit in logits, convert it to prob using softmax

softmax_logits = [softmax(logit, dim=-1) for logit in logits]
print(softmax_logits)

In [ ]:
# get the index of max prob for each logit
max_prob_index = [torch.argmax(softmax_logit, dim=-1) for softmax_logit in softmax_logits]
print(max_prob_index)

In [ ]:
# from the softmax_logits, get the indices of top 10 max prob 
top_10_indices = [torch.topk(softmax_logit, k=10, dim=-1) for softmax_logit in softmax_logits]
print(top_10_indices)

In [ ]:
# from the top_10_indices, get the words in Hindi vocab using tokenizer.target_tokenizer
word_list = []
for i in top_10_indices:
    with en_indic_tokenizer.as_target_tokenizer():
        word_list.append(en_indic_tokenizer.batch_decode(i.indices))
print(word_list)

In [ ]:
for _ in word_list:
    print(_)